# A primer on data visualization in research

## A simple gene expression study

In this visualization example we will create a volcano plot in order to find up-regulated and down-regulated genes based on a set of gene expression data. The fold change is pre-computed based on the expression data, and included in the dataset.

The provided dataset is currently being used in a pharmacogenomics study in order to evaluate the effect of DNA regulation on drug response.

### Import pandas and altair

Activate the relevant libraries for the analysis.

If you get an error, try running the following code in the command line:
```bash
python3 -m pip install pandas
python3 -m pip install altair[all]
```

If this doesn't work you might have to install the packages using `python` instead of `python3` in the above commands.

In case of emergency, add the following commands to the cell below:
```python
%pip install pandas
%pip install altair[all]
```

In [ ]:
import pandas as pd
import altair as alt
import math
import os

alt.data_transformers.enable("vegafusion")

### Load the dataset into a dataframe

We start by creating our dataframe from the provided `.csv` data. Note that we are only inspecting the top 5 entries, since our dataframe consists of 12621 rows!

In [ ]:
df = pd.read_csv(os.path.realpath("../data/raw/gene-expression-data.csv"))
df.head()

### Data analysis

We first perform our data analysis. We compare our dataframe to certain thresholds in order to define which genes are differentially expressed, and thus expressed more (upregulated) or expressed less (downregulated) than normally expected. This is calculated based on our `log10(foldchange)` (the `logFC` column), which expresses the difference in expression in comparison with the baseline.

#### Define thresholds

We use a simple confidence threshold of $0.05$. Statistically speaking we would need to perform a correction (something like a [Bonferroni correction](https://en.wikipedia.org/wiki/Bonferroni_correction)), since we are performing multiple statistical tests, but we'll disregard that for now.

We are also using an arbitrarily defined threshold of $0.6$ for our fold-change.

In [ ]:
CONFIDENCE = 0.05
REGULATION = 0.6

#### Perform comparisons with the defined thresholds

In [ ]:
df["expression"] = "NORMAL"
df.loc[(df["adj.P.Val"] < CONFIDENCE) & (df["logFC"] > REGULATION), "expression"] = "UP"
df.loc[(df["adj.P.Val"] < CONFIDENCE) & (df["logFC"] < -REGULATION), "expression"] = "DOWN"

df.head()

### Data visualization

We start by visualizing our data in a normal scatter plot.

In [ ]:
scatter = alt.Chart(df).mark_circle(size=60).encode(
    x='logFC',
    y='adj.P.Val',
).interactive()

scatter

In [ ]:
# Check our data to see why our result is weird
df.describe()

As you can see, we are getting no results. This is because of the range of the `adj.P.Val` column (in relation to the `logFC` column). We need to correct for this by taking the $-log10$ of our p-values.

In [ ]:
df["logp"] = -df["adj.P.Val"].apply(math.log10)
df.head()

In [ ]:
# Better!
df.describe()

Now we are ready to visualize our data in a very standard volcano plot.

In [ ]:
scatter = alt.Chart(df).mark_circle(size=60).encode(
    x='logFC',
    y='logp',
).interactive()

scatter

### Interpreting the data

It's time to apply our thresholds to the visualization. Let's start by plotting the thresholds in our volcano plot as lines in order to visualize what we are doing.

Note that we are taking the $-log10$ of our p-value threshold, since it has to be adjusted to our new scaling.

In [ ]:
pval = alt.Chart(pd.DataFrame({'y': [-math.log10(CONFIDENCE)]})).mark_rule(color="red").encode(y='y')

updownreg = alt.Chart(pd.DataFrame({'x': [-REGULATION, REGULATION]})).mark_rule(color="red").encode(x='x')

scatter_thresholds = scatter + pval + updownreg
scatter_thresholds

It's more intuitive to show our thresholds by using colours instead of lines. The following code maps our `expression` column to either a red (upregulated), black (normal) or blue (downregulated) colour.

In [ ]:
domain = df["expression"].unique()
range_ = ["red", "blue", "black"]

volcano = alt.Chart(df).mark_circle(size=60).encode(
    x='logFC',
    y='logp',
    color=alt.Color('expression', scale=alt.Scale(domain=domain, range=range_)).title("Gene regulation"),
).interactive()
volcano

### Final visualization and drawing conclusions

We will finalize our visualization by adding names to the five most upregulated and downregulated genes. We will use this to draw our conclusions.

First we calculate which genes we need, and add their names to a separate column. This column only contains a name for our found genes, and nothing (an empty string as it's called `""`, this represents the absence of text) for the other genes.

In [ ]:
# Define the amount of hits we are looking for
N_TOP_HITS = 5

# Find the five most upregulated genes
max_up = df.loc[df["expression"] == "UP", ["Gene.symbol", "logFC"]]
max_up.sort_values("logFC", ascending=False, inplace=True)
max_up = list(max_up["Gene.symbol"][:N_TOP_HITS])

# Find the five most downregulated genes
max_down = df.loc[df["expression"] == "DOWN", ["Gene.symbol", "logFC"]]
max_down.sort_values("logFC", ascending=True, inplace=True)
max_down = list(max_down["Gene.symbol"][:N_TOP_HITS])

# Only add labels for the found genes
def filter_labels(genename):
    if genename in max_up or genename in max_down:
        return genename
    else:
        return ""

df["label"] = df["Gene.symbol"].apply(filter_labels)
df.head()

Then we add this new column to our visualization as a label for our datapoints.

In [ ]:
# Apply the labels to our plot
text = volcano.mark_text(
    align='left',
    baseline='middle',
    dx=7
).encode(
    text='label'
)

volcano + text

Using our volcano plot, we can conclude that PYGM is the most downregulated gene, while MMP1 is the most upregulated gene.